In [1]:
import os
import pandas as pd
import numpy as np
import math

import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation, Dense, Flatten
import tensorflow.python.keras.optimizers
import tensorflow.python.keras.metrics

import keras_tuner as kt

#%%capture
data_file_path = os.path.join(os.getcwd(), "Data\\returnsData.csv")
returnsData = pd.read_csv(data_file_path)
#returnsData.info(verbose=True)
longReturns = returnsData[["YearQuarter", "gvkey", "quarterlyReturns"]] #"quarterlyVolatility",
#longReturns.info(verbose=True)
wideReturns = pd.pivot(longReturns, index="YearQuarter", columns="gvkey", values="quarterlyReturns")
#wideReturns.info(verbose=True)
#print(wideReturns.iloc[0:10,0:10])

#wideReturns["YearQuarter"] = wideReturns.index
#wideReturns["YQ"] = wideReturns.index

trainSize = math.floor(len(wideReturns.index) * 0.5)
validationSize = math.floor(len(wideReturns.index) * 0.75)
testSize = len(wideReturns.index)
# trainSize = len(wideReturns["YearQuarter"]) * 0.5
# validationSize = len(wideReturns["YearQuarter"]) * 0.75
#wideReturns[:,"Split"] = 


sample_length = 10

#def preprocessData(startOfSplit, endOfSplit):
dataset = tf.keras.preprocessing.timeseries_dataset_from_array(
    data=wideReturns, targets=None,
    sequence_length=sample_length, sequence_stride=1, sampling_rate=1,
    batch_size=128,  shuffle=False, 
    start_index=startOfSplit, end_index=endOfSplit)

for batch in dataset:
  data = batch
  #assert np.array_equal(inputs[0], wideReturns[:sample_length])
  # second sample equals output timestamps 20-40
  #assert np.array_equal(targets[1], wideReturns[sample_length:2*sample_length])
  break

print(data.shape)
# print(dataset.cardinality())
# print(dataset.cache())

# xTrain = wideReturns.iloc[:(trainSize-sample_length),:]
# yTrain = wideReturns.iloc[sample_length:trainSize,:]

# xVal = wideReturns.iloc[(trainSize-sample_length):(validationSize-sample_length),:]
# yVal = wideReturns.iloc[trainSize:validationSize,:]

# xTest = wideReturns.iloc[validationSize-sample_length:-sample_length,:]
# yTest = wideReturns.iloc[validationSize:,:]

# xTrainReshaped = np.reshape(xTrain.to_numpy(), (-1, 1), order="F")
# yTrainReshaped = np.reshape(yTrain.to_numpy(), (-1, 1), order="F")
# xValReshaped = np.reshape(xVal.to_numpy(), (-1, 1), order="F")
# yValReshaped = np.reshape(yVal.to_numpy(), (-1, 1), order="F")
# xTestReshaped = np.reshape(xTest.to_numpy(), (-1, 1), order="F")
# yTestReshaped = np.reshape(yTest.to_numpy(), (-1, 1), order="F")


#[lags of YearQuarter, YearQuarter, gvkey] => [Batch, YearQuarter, None]
reshapedData = np.transpose(a=data, axes=(2, 0, 1))
reshapedData = np.reshape(reshapedData, (-1, sample_length))
reshapedData = reshapedData[~np.isnan(reshapedData).any(axis=1),:]
reshapedData = np.expand_dims(reshapedData, axis=2)
#return reshapedData

#scaledData = normlayer(reshapedData)

# train = preprocessData(None, trainSize)
# val = preprocessData(trainSize, validationSize)
# test = preprocessData(validationSize, None)

# xTrain, xVal, xTest = train[:,:-1,:], val[:,:-1,:], test[:,:-1,:]
# yTrain, yVal, yTest = train[:,-1,:], val[:,-1,:], test[:,-1,:]

normlayer = tf.keras.layers.Normalization(axis=None)
#normlayer.adapt(xTrain[:,0,:])
normlayer.adapt(wideReturns[trainSize,:])



In [ ]:
d = data.iloc[:(trainSize-sample_length),:]

In [ ]:
sample_length=10
dataset = tf.keras.preprocessing.timeseries_dataset_from_array(
    data=wideReturns.iloc[:-sample_length,:], targets=wideReturns.iloc[sample_length:, :],
    sequence_length=sample_length, sequence_stride=1, sampling_rate=1,
    batch_size=8,  shuffle=False, 
    start_index=None, end_index=None)

for batch in dataset:
    data, dataY = batch
    #assert np.array_equal(inputs[0], wideReturns[:sample_length])
    # second sample equals output timestamps 20-40
    #assert np.array_equal(targets[1], wideReturns[sample_length:2*sample_length])
    break

trainSize = math.floor(data.shape[0] * 0.5)
validationSize = math.floor(data.shape[0] * 0.75)

xTrain, xVal, xTest = data[:trainSize,:-1,:], data[trainSize:validationSize,:-1,:], data[validationSize:,:-1,:]
yTrain, yVal, yTest = data[:trainSize,-1,:], data[trainSize:validationSize,-1,:], data[validationSize:,-1,:]

normlayer = tf.keras.layers.Normalization(axis=None)
normlayer.adapt(xTrain[:,0,:])
#normlayer.adapt(wideReturns.iloc[:trainSize,:])


In [ ]:
a=wideReturns.iloc[:-sample_length,:]
b=wideReturns.iloc[sample_length:, :]

In [ ]:

trainSize = math.floor(len(wideReturns.index) * 0.5)
validationSize = math.floor(len(wideReturns.index) * 0.75)
testSize = len(wideReturns.index)

xTrain = wideReturns.iloc[:(trainSize-sample_length),:]
yTrain = wideReturns.iloc[sample_length:trainSize,:]

xVal = wideReturns.iloc[(trainSize-sample_length):(validationSize-sample_length),:]
yVal = wideReturns.iloc[trainSize:validationSize,:]

xTest = wideReturns.iloc[validationSize-sample_length:-sample_length,:]
yTest = wideReturns.iloc[validationSize:,:]

xTrainReshaped = np.reshape(xTrain.to_numpy(), (-1, 1), order="F")
yTrainReshaped = np.reshape(yTrain.to_numpy(), (-1, 1), order="F")
xValReshaped = np.reshape(xVal.to_numpy(), (-1, 1), order="F")
yValReshaped = np.reshape(yVal.to_numpy(), (-1, 1), order="F")
xTestReshaped = np.reshape(xTest.to_numpy(), (-1, 1), order="F")
yTestReshaped = np.reshape(yTest.to_numpy(), (-1, 1), order="F")

batch_size=1

train = tf.keras.preprocessing.timeseries_dataset_from_array(
    data=xTrainReshaped, targets=yTrainReshaped,
    sequence_length=sample_length, sequence_stride=1, sampling_rate=1,
    batch_size=batch_size,  shuffle=False)
validation = tf.keras.preprocessing.timeseries_dataset_from_array(
    data=xValReshaped, targets=yValReshaped,
    sequence_length=sample_length, sequence_stride=1, sampling_rate=1,
    batch_size=batch_size,  shuffle=False)
test = tf.keras.preprocessing.timeseries_dataset_from_array(
    data=xTestReshaped, targets=yTestReshaped,
    sequence_length=sample_length, sequence_stride=1, sampling_rate=1,
    batch_size=batch_size,  shuffle=False)


# train = tf.keras.preprocessing.timeseries_dataset_from_array(
#     data=xTrainReshaped, targets=yTrainReshaped,
#     sequence_length=trainSize, sequence_stride=sample_length, sampling_rate=1,
#     batch_size=batch_size,  shuffle=False)
# validation = tf.keras.preprocessing.timeseries_dataset_from_array(
#     data=xValReshaped, targets=yValReshaped,
#     sequence_length=validationSize-trainSize, sequence_stride=sample_length, sampling_rate=1,
#     batch_size=batch_size,  shuffle=False)
# test = tf.keras.preprocessing.timeseries_dataset_from_array(
#     data=xTestReshaped, targets=yTestReshaped,
#     sequence_length=testSize-validationSize, sequence_stride=sample_length, sampling_rate=1,
#     batch_size=batch_size,  shuffle=False)




a = wideReturns.iloc[...,:15]
b = list(train.take(10).as_numpy_iterator())
#c = train.as_numpy_iterator()#[...,:10]
list(train.take(1).as_numpy_iterator())


In [ ]:
#wideReturns.head()
#a = wideReturns.iloc[:10, :10]

wideReturns.info(verbose=True)

In [ ]:
model = Sequential()
model.add(normlayer)
model.add(tf.keras.layers.LSTM(units=5)) #, input_shape=(,sample_length-1)   #mask
model.add(Dense(units=1))

model.compile(optimizer="adam", #tf.keras.optimizers.Adam(learning_rate=0.001)
              loss="MeanAbsoluteError",
              metrics=[tf.keras.losses.MeanSquaredError(), tf.keras.losses.MeanAbsoluteError()])

#model.fit(x=xTrain, y=yTrain, batch_size=32, validation_data=(xVal, yVal))
model.fit(x=dataset, batch_size=8)#, validation_data=(xVal, yVal))

# class MyHyperModel(kt.HyperModel):
#     def build(self, hp):
#         model = Sequential()
#         model.add(tf.keras.layers.LSTM(units=hp.Int("units", min_value=32, max_value=512, step=32), input_shape=(sample_length-1,)))
#         model.add(Dense(units=1))

#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=hp.Int("learning_rate", [16, 32])),
#                   loss="MeanAbsoluteError",
#                   metrics=[tf.keras.losses.MeanSquaredError(), tf.keras.losses.MeanAbsoluteError()])
#         return model

#     def fit(self, hp, model, *args, **kwargs):
#         return model.fit(
#             *args,
#             batch_size=hp.Choice("batch_size", [16, 32]),
#             **kwargs,
#         )

# tuner = kt.RandomSearch(
#     MyHyperModel(),
#     objective="val_accuracy",
#     max_trials=3,
#     overwrite=True,
#     directory="my_dir",
#     project_name="tune_hypermodel",
# )